# IS Assignment 1
## Ayaan Danish 20K-0245
## DES Algorithm Implementation 

In [149]:
# Class to create block of bits as a string. These are what we will work with when applying DES.
class BitString:
    # Constructor takes in an input string, and stores it in ASCII Binary form
    # All bit strings are padded to a block size of 64 using trailing 0s
    def __init__(self, input_string):
        self.binary = ''.join(format(ord(char), '08b') for char in input_string).ljust(64, '0')

    def __str__(self):
        return self.binary
    
    def __repr__(self):
        return self.binary

    # Divides the bit string in half, circular shifts to the left by the given amount to each half, and merges the halves back
    def left_shift(self, shift_amount):
        shift_amount = shift_amount % len(self.binary)
        first_half = self.binary[:len(self.binary)//2]
        second_half = self.binary[len(self.binary)//2:]

        first_half = first_half[shift_amount:] + first_half[:shift_amount]
        second_half = second_half[shift_amount:] + second_half[:shift_amount]
        
        self.binary = first_half + second_half
        return self
        
    # Returns the textual form of the bit string
    def to_text(self):
        # Read the bit string 8 bits at a time and convert them to characters. Remove trailing '00000000's at the end (undo padding)
        return ''.join([chr(int(self.binary[i:i+8], 2)) for i in range(0, len(self.binary), 8)]).strip(chr(int("00000000", 2)))
    
    # Returns the hexadecimal form of the bit string
    def to_hex(self):
        return ''.join(format(int(self.binary[i:i+4], 2), 'x') for i in range(0, len(self.binary), 4)).strip(format(int('00000000', 2), 'x'))

In [154]:
# Class for the DES Algorithm that holds all the data (keys and plaintext) and methods (DES operations)
class DES:
    # Constructor takes in a plain text and a key as a string of any length
    def __init__(self, plain, key):
        # Key is fixed at 8 characters (64 bits)
        self.cipher_key = BitString(key[:8])
        # Plain text is broken into blocks of 64 bits each, padded with 0 bits
        self.encryption_text = [BitString(plain[i:i+8]) for i in range(0, len(plain), 8)]
        self.round_keys = []

    # Function to perform 8 bit parity drop on the cipher key during key-gen (64 bits to 56 bits)
    def parity_drop(self):
        parity_drop_table = [
            57, 49, 41, 33, 25, 17, 9, 1,
            58, 50, 42, 34, 26, 18, 10, 2,
            59, 51, 43, 35, 27, 19, 11, 3,
            60, 52, 44, 36, 63, 55, 47, 39,
            31, 23, 15, 7, 62, 54, 46, 38,
            30, 22, 14, 6, 61, 53, 45, 37,
            29, 21, 13, 5, 28, 20, 12, 4
        ]     
        self.cipher_key.binary = ''.join(self.cipher_key.binary[parity_drop_table[i]-1] for i, item in enumerate(parity_drop_table))

    # Performs a Compression P-Box operation on the input and returns the output
    def compression_p_box(self, input):
        compression_p_table = [
            14, 17, 11, 24, 1, 5, 3, 28,
            15, 6, 21, 10, 23, 19, 12, 4,
            26, 8, 16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55, 30, 40,
            51, 45, 33, 48, 44, 49, 39, 56,
            34, 53, 46, 42, 50, 36, 29, 32
        ]
        return ''.join(input[compression_p_table[i]-1] for i, item in enumerate(compression_p_table))
    
    # Performs an expansion P-Box operation on the input and returns the output
    def expansion_p_box(self, input):
        expansion_p_table = [
            32, 1, 2, 3, 4, 5,
            4, 5, 6, 7, 8, 9, 
            8, 9, 10, 11, 12, 13,
            12, 13, 14, 15, 16, 17,
            16, 17, 18, 19, 20, 21,
            20, 21, 22, 23, 24, 25,
            24, 25, 26, 27, 28, 29,
            28, 29, 30, 31, 32, 1
        ]
        return ''.join(input[expansion_p_table[i]-1] for i, item in enumerate(expansion_p_table))
    
    # Performs a straight P-Box operation on the input and returns the output
    def straight_p_box(self, input):
        straight_p_table = [
            16, 7, 20, 21, 29, 12, 28, 17,
            1, 15, 23, 26, 5, 18, 31, 10,
            2, 8, 24, 14, 32, 27, 3, 9,
            19, 13, 30, 6, 22, 11, 4, 25
        ]
        return ''.join(input[straight_p_table[i]-1] for i, item in enumerate(straight_p_table))

    # Function to generate the round keys (populates the round_keys list)
    def generate_round_keys(self):
        # First perform parity drop
        self.parity_drop()
        # Then for each round...
        for round_num in range(1, 17):
            # Break the key (56 bits) into two halves (28-28 bits), left shift, and re-merge
            self.cipher_key.left_shift(1 if round_num in [1, 2, 9, 16] else 2)
            #Then perform a Compression P-Box to get the round key (48 bits)
            self.round_keys.append(self.compression_p_box(self.cipher_key.binary))

    # Performs the initial permutation on the plain text, before the first round
    def initial_permutation(self, block_num):
        initial_p_table = [
            58, 50, 42, 34, 26, 18, 10, 2,
            60, 52, 44, 36, 28, 20, 12, 4,
            62, 54, 46, 38, 30, 22, 14, 6,
            64, 56, 48, 40, 32, 24, 16, 8,
            57, 49, 41, 33, 25, 17, 9, 1,
            59, 51, 43, 35, 27, 19, 11, 3,
            61, 53, 45, 37, 29, 21, 13, 5,
            63, 55, 47, 39, 31, 23, 15, 7
        ]
        self.encryption_text[block_num].binary = ''.join(self.encryption_text[block_num].binary[initial_p_table[i]-1] for i, item in enumerate(initial_p_table))
    
    # Performs the final permutation on the plain text, after the final round
    def final_permutation(self, block_num):
        final_p_table = [
            40, 8, 48, 16, 56, 24, 64, 32,
            39, 7, 47, 15, 55, 23, 63, 31,
            38, 6, 46, 14, 54, 22, 62, 30,
            37, 5, 45, 13, 53, 21, 61, 29,
            36, 4, 44, 12, 52, 20, 60, 28,
            35, 3, 43, 11, 51, 19, 59, 27,
            34, 2, 42, 10, 50, 18, 58, 26,
            33, 1, 41, 9, 49, 17, 57, 25
        ]
        self.encryption_text[block_num].binary = ''.join(self.encryption_text[block_num].binary[final_p_table[i]-1] for i, item in enumerate(final_p_table))

    # Function that returns the output (4 bits) for a given S-Box (1-8) for a given input (6 bits)
    def s_box(self, input, box_num):
        s_box_tables = [
            {
                0: [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
                1: [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
                2: [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
                3: [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
            },
            {
                0: [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
                1: [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
                2: [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
                3: [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
            },
            {
                0: [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
                1: [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
                2: [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
                3: [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
            },
            {
                0: [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
                1: [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
                2: [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
                3: [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
            },
            {
                0: [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
                1: [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 6, 9, 11, 5],
                2: [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
                3: [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 3, 5, 10, 4]
            },
            {
                0: [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
                1: [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
                2: [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
                3: [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
            },
            {
                0: [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
                1: [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
                2: [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
                3: [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
            },
            {
                0: [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
                1: [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
                2: [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
                3: [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
            }
        ]

        # Calculate the row for looking up the S-Box table using the first and last bits
        row = int(input[0] + input[-1], 2)
        # Calculate the col for looking up the S-Box table using the middle 4 bits
        col = int(input[1:-1], 2)
        
        return bin(s_box_tables[box_num][row][col])[2:].rjust(4, '0')

    # Function to break the input (48 bits) into 8 pieces of 6 bits, and get the final S-Box output (32 bits)
    def apply_s_boxes(self, input):
        box_inputs = [input[i*6: 6+i*6] for i in range(8)]
        box_outputs = [self.s_box(input=item, box_num=num) for num, item in enumerate(box_inputs)]
        return ''.join(box_outputs)
    
    # Function to use the above operations to apply the Feistel Function to the right side of the plain text
    def feistel_function(self, input, round_key):
        # First perform an expansion P-Box on the input (32 bits) to get the output (48 bits)
        input = self.expansion_p_box(input)
        # XOR the input (48 bits) with the key for this round (48 bits) 
        input = ''.join('1' if bit_1 != bit_2 else '0' for bit_1, bit_2 in zip(input, round_key))
        # Apply the S-Boxes to the input (48 bits) to get the output (32 bits)
        input = self.apply_s_boxes(input)
        # Apply a straight P-Box to the input to get the output of the Feistel Function
        input = self.straight_p_box(input)
        return input

    # Function to perform a single round of DES encryption to the plain text, given the round key
    def encryption_round(self, block_num, round_key, is_final_round):
        # First, split the plain text into 2 halves
        left_half = self.encryption_text[block_num].binary[:len(self.encryption_text[block_num].binary)//2]
        right_half = self.encryption_text[block_num].binary[len(self.encryption_text[block_num].binary)//2:]

        # Apply the Feistel Function to the right half and XOR it with the left half
        processed_half = self.feistel_function(right_half, round_key)
        left_half = ''.join('1' if bit_1 != bit_2 else '0' for bit_1, bit_2 in zip(left_half, processed_half))
        

        # Swap the left and right halves with each other
        if not is_final_round:
            left_half, right_half = right_half, left_half

        # Merge the left and right half back, giving the result of this encryption round
        self.encryption_text[block_num].binary = ''.join((left_half, right_half))

    # Function to perform the DES encryption
    def encrypt(self):
        # First, generate the round keys
        self.generate_round_keys()
        
        for block_num, block in enumerate(self.encryption_text):
        # Perform the initial permutation
            self.initial_permutation(block_num)
            
            # Then apply 16 rounds of encryption to the plaintext using the round keys
            for round_num in range(1, 17):
                round_key = self.round_keys[round_num-1]
                self.encryption_round(block_num, round_key, is_final_round=round_num == 16)
            
            # Finally, apply the final permutation to get the final cipher text
            self.final_permutation(block_num)

        print('Plain Text has been encrypted!')

    # Function to decrypt the cipher text
    def decrypt(self):
        if self.round_keys == []:
            print('You have not encrypted anything yet!')
            return

        for block_num, block in enumerate(self.encryption_text):
            self.initial_permutation(block_num)

            for round_num in reversed(range(1, 17)):
                round_key = self.round_keys[round_num-1]
                self.encryption_round(block_num, round_key, is_final_round=round_num == 1)

            self.final_permutation(block_num)

        print('Cipher text has been decrypted!')
    
    def get_text(self):
        return ''.join(block.to_text() for block in self.encryption_text)
    

In [151]:
plain = input('Enter your plain text: ')
key = input('Enter your key: ')

print(f'Your Plain Text: {plain}')
print(f'Your Key Text: {key}')

Your Plain Text: Hello there my name is Syed Ayaan Danish i am encrypting this text
Your Key Text: key


In [156]:
des = DES(plain, key)
des.encrypt()
print(f'Cipher Text: {des.get_text()}')
des.decrypt()
print(f'Plain Text: {des.get_text()}')

Plain Text has been encrypted!
¡ë?ëÃxidÙ\  ±0âÌÀ-ãzs©/ó·G!ôÃ ¼¾}C`¿Úpû,ßÍ»ÌJUS£XÉ
Cipher text has been decrypted!
Plain Text: Hello there my name is Syed Ayaan Danish i am encrypting this text
